<a href="https://colab.research.google.com/github/xiomara-udea/COMPETENCIA-AI4ENG-UDEA---Semestre-2025-1/blob/main/04_modelo_con_preprocesado_con_RandomizedSearchCV_y_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"xiomaraperezudea","key":"c7779ed01610d092aea8f2d5f696098f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 684MB/s]


In [ ]:
!unzip -l udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
  4716673  2025-03-25 16:54   submission_example.csv
 59185250  2025-03-25 16:54   test.csv
143732449  2025-03-25 16:54   train.csv
---------                     -------
207634372                     3 files


In [ ]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
dtr = pd.read_csv("train.csv")
print (dtr.shape)
dts = pd.read_csv("test.csv")
print (dts.shape)

(692500, 21)
(296786, 20)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

class SaberProCleaner:
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.columnas_onehot = ['PERIODO', 'ESTU_PRGM_DEPARTAMENTO', 'AREA_ACADEMICA']
        self.columns_after_onehot = None
        self.columnas_a_normalizar = ['ESTU_VALORMATRICULAUNIVERSIDAD_num', 'ESTU_HORASSEMANATRABAJA_num']

        self.mapa_valormatricula = {
            'Menos de 500 mil': 250_000,
            'Entre 500 mil y menos de 1 millón': 750_000,
            'Entre 1 millón y menos de 2.5 millones': 1_750_000,
            'Entre 2.5 millones y menos de 4 millones': 3_250_000,
            'Entre 4 millones y menos de 5.5 millones': 4_750_000,
            'Entre 5.5 millones y menos de 7 millones': 6_250_000,
            'Más de 7 millones': 8_000_000,
            'No pagó matrícula': 0
        }

        self.mapa_horas_trabajo = {
            '0': 0,
            'Menos de 10 horas': 5,
            'Entre 10 y 20 horas': 15,
            'Entre 21 y 30 horas': 25,
            'Más de 30 horas': 35
        }

        self.binarios_bool = {
            'FAMI_TIENEINTERNET': {'Si': True, 'No': False},
            'FAMI_TIENELAVADORA': {'Si': True, 'No': False},
            'FAMI_TIENEAUTOMOVIL': {'Si': True, 'No': False},
            'ESTU_PRIVADO_LIBERTAD': {'S': True, 'N': False},
            'ESTU_PAGOMATRICULAPROPIO': {'Si': True, 'No': False},
            'FAMI_TIENECOMPUTADOR': {'Si': True, 'No': False}
        }

        self.educacion_orden = {
            'Ninguno': 1,
            'Primaria incompleta': 2,
            'Primaria completa': 3,
            'Secundaria (Bachillerato) incompleta': 4,
            'Secundaria (Bachillerato) completa': 5,
            'Técnica o tecnológica incompleta': 6,
            'Técnica o tecnológica completa': 7,
            'Educación profesional incompleta': 8,
            'Educación profesional completa': 9,
            'Postgrado': 10
        }

        self.rendimiento_orden = {
            'bajo': 1,
            'medio-bajo': 2,
            'medio-alto': 3,
            'alto': 4
        }

    def limpiar_programa(self, texto):
        if pd.isnull(texto): return texto
        texto = texto.upper().strip()
        texto = (texto.replace('Á','A').replace('É','E').replace('Í','I')
                      .replace('Ó','O').replace('Ú','U').replace('Ñ','N')
                      .replace('¿','').replace('¡','').replace('.', '')
                      .replace('-', ' ').replace('_', ' '))
        while '  ' in texto:
            texto = texto.replace('  ', ' ')
        return texto

    def clasificar_area(self, programa):
        programa = programa.upper()
        if any(p in programa for p in ['INGENIERIA', 'INGENIERÍA']):
            return 'Ingenierías'
        elif any(p in programa for p in ['MEDICINA', 'ENFERMERIA', 'SALUD', 'FISIOTERAPIA', 'ODONTOLOGIA', 'NUTRICION', 'TERAPIA']):
            return 'Salud'
        elif any(p in programa for p in ['DERECHO']):
            return 'Derecho'
        elif any(p in programa for p in ['ADMINISTRACION', 'CONTADURIA', 'ECONOMIA']):
            return 'Económico-administrativas'
        elif any(p in programa for p in ['BIOLOGIA', 'FISICA', 'QUIMICA', 'MATEMATICA']):
            return 'Ciencias Básicas'
        elif any(p in programa for p in ['PSICOLOGIA', 'SOCIOLOGIA', 'TRABAJO SOCIAL']):
            return 'Ciencias Sociales y Humanas'
        elif any(p in programa for p in ['ARTES', 'DISEÑO', 'MUSICA']):
            return 'Artes'
        elif any(p in programa for p in ['LICENCIATURA', 'EDUCACION']):
            return 'Educación'
        elif any(p in programa for p in ['COMUNICACION']):
            return 'Comunicación'
        elif any(p in programa for p in ['AGRONOMIA', 'VETERINARIA']):
            return 'Agropecuarias y Medio Ambiente'
        else:
            return 'Otras'

    def fit_transform(self, df):
        return self._transform(df.copy(), fit_scaler=True)

    def transform(self, df):
        return self._transform(df.copy(), fit_scaler=False)

    def _transform(self, df, fit_scaler):
        # 1. Drop columna duplicada
        df = df.drop(columns=['FAMI_TIENEINTERNET.1'], errors='ignore')

        # 2. Mapeo ordinal
        df['ESTU_VALORMATRICULAUNIVERSIDAD_num'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(self.mapa_valormatricula)
        df['ESTU_HORASSEMANATRABAJA_num'] = df['ESTU_HORASSEMANATRABAJA'].map(self.mapa_horas_trabajo)
        df['ESTU_VALORMATRICULAUNIVERSIDAD_num'] = df['ESTU_VALORMATRICULAUNIVERSIDAD_num'].fillna(df['ESTU_VALORMATRICULAUNIVERSIDAD_num'].mean())
        df['ESTU_HORASSEMANATRABAJA_num'] = df['ESTU_HORASSEMANATRABAJA_num'].fillna(df['ESTU_HORASSEMANATRABAJA_num'].mean())

        # 3. Relleno por ffill
        columnas_ffill = [
            'FAMI_ESTRATOVIVIENDA',
            'FAMI_TIENEINTERNET',
            'FAMI_EDUCACIONPADRE',
            'FAMI_TIENELAVADORA',
            'FAMI_TIENEAUTOMOVIL',
            'ESTU_PAGOMATRICULAPROPIO',
            'FAMI_TIENECOMPUTADOR',
            'FAMI_EDUCACIONMADRE'
        ]
        for col in columnas_ffill:
            if col in df.columns:
                df[col] = df[col].ffill()

        # 4. Programa y área
        df['PROGRAMA_LIMPIO'] = df['ESTU_PRGM_ACADEMICO'].apply(self.limpiar_programa)
        df['AREA_ACADEMICA'] = df['PROGRAMA_LIMPIO'].apply(self.clasificar_area)
        df.drop(columns=['ESTU_PRGM_ACADEMICO', 'PROGRAMA_LIMPIO'], inplace=True)

        # 5. Mapear booleanos
        for col, mapping in self.binarios_bool.items():
            if col in df.columns:
                df[col] = df[col].map(mapping).astype(bool)

        # 6. Codificación ordinal
        for col in ['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']:
            df[col] = df[col].replace(['Ninguno', 'No sabe', 'No Aplica'], np.nan).ffill()
            df[col + '_ord'] = df[col].map(self.educacion_orden)

        if 'RENDIMIENTO_GLOBAL' in df.columns:
            df['RENDIMIENTO_GLOBAL_ord'] = df['RENDIMIENTO_GLOBAL'].map(self.rendimiento_orden)
            df.drop(columns=['RENDIMIENTO_GLOBAL'], inplace=True)

        df.drop(columns=[
            'FAMI_ESTRATOVIVIENDA',
            'FAMI_EDUCACIONPADRE',
            'FAMI_EDUCACIONMADRE',
            'ESTU_VALORMATRICULAUNIVERSIDAD',
            'ESTU_HORASSEMANATRABAJA'
        ], errors='ignore', inplace=True)

        # 7. One-hot
        df = pd.get_dummies(df, columns=self.columnas_onehot, dtype=bool)

        # Guardar columnas para usar en test
        if fit_scaler:
            self.columns_after_onehot = df.columns.tolist()

        # Asegurar columnas iguales en test
        if not fit_scaler:
            for col in self.columns_after_onehot:
                if col not in df:
                    df[col] = False
            df = df[self.columns_after_onehot]

        # 8. Escalar
        scaled = self.scaler.fit_transform(df[self.columnas_a_normalizar]) if fit_scaler else self.scaler.transform(df[self.columnas_a_normalizar])
        df_scaled = pd.DataFrame(scaled, columns=[c + '_scaled' for c in self.columnas_a_normalizar], index=df.index)
        df = pd.concat([df, df_scaled], axis=1)
        df.drop(columns=self.columnas_a_normalizar, inplace=True)

        return df


In [ ]:
# Cargar y limpiar los datos

# Entrenamiento
cleaner = SaberProCleaner()
dtr_limpio = cleaner.fit_transform(dtr)

# Prueba
dts_limpio = cleaner.transform(dts)



In [ ]:
# =========================================
# 3. SEPARAR FEATURES Y ETIQUETAS
# =========================================
Xtr = dtr_limpio.drop(columns=["RENDIMIENTO_GLOBAL_ord"]).values
ytr = dtr_limpio["RENDIMIENTO_GLOBAL_ord"].values
Xts = dts_limpio.values

# Ajustar etiquetas para XGBoost (de 1–4 → 0–3)
ytr_xgb = ytr - 1


In [ ]:
# =========================================
# 4. MODELO XGBOOST + CROSS VALIDATION
# =========================================
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    eval_metric='mlogloss',
    random_state=42
)


scores = cross_val_score(xgb_model, Xtr, ytr_xgb, cv=5, scoring='accuracy')
print("Accuracy promedio (XGBoost, CV 5):", scores.mean())


Accuracy promedio (XGBoost, CV 5): 0.42039711191335744


In [ ]:
# =========================================
# 5. ENTRENAMIENTO FINAL Y

# =========================================
xgb_model.fit(Xtr, ytr_xgb)
preds_ts = xgb_model.predict(Xts)
preds_ts = preds_ts + 1  # Volver a rango 1-4

# =========================================
# 6. CONVERTIR PREDICCIONES A TEXTO
# =========================================
inv_rendimiento_orden = {
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
}
preds_ts_labels = [inv_rendimiento_orden[p] for p in preds_ts]

# =========================================
# 7. CREAR ARCHIVO DE SUBMISSION
# =========================================
submission = pd.DataFrame({
    'ID': dts['ID'],
    'RENDIMIENTO_GLOBAL': preds_ts_labels
})
submission.to_csv("submission_xgboost.csv", index=False)
submission.head()